In [1]:
import numpy as np
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.ppo import MlpPolicy
from imitation.algorithms import density as db
from imitation.algorithms.adversarial.airl import AIRL
from imitation.data import rollout
from imitation.data.wrappers import RolloutInfoWrapper
from imitation.policies.serialize import load_policy
from imitation.rewards.reward_nets import BasicShapedRewardNet
from imitation.util.networks import RunningNorm
from imitation.util.util import make_vec_env
from imitation.data.types import TrajectoryWithRew
import numpy as np
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.ppo import MlpPolicy

from imitation.algorithms.adversarial.gail import GAIL
from imitation.data import rollout
from imitation.data.wrappers import RolloutInfoWrapper
from imitation.policies.serialize import load_policy
from imitation.rewards.reward_nets import BasicRewardNet
from imitation.util.networks import RunningNorm
from imitation.util.util import make_vec_env

from stk_actor.wrappers import PreprocessObservationWrapper

SEED = 42

from stable_baselines3 import PPO, A2C
from stable_baselines3.common.env_util import make_vec_env
import gymnasium as gym
from pystk2_gymnasium import AgentSpec
from bbrl.agents.gymnasium import ParallelGymAgent, make_env
from functools import partial

noob_vec_env = make_vec_env("supertuxkart/flattened_multidiscrete-v0", seed=0, n_envs=21, wrapper_class=lambda x : (PreprocessObservationWrapper(x, ret_dict=False)), env_kwargs={
    'render_mode':None, 'agent':AgentSpec(use_ai=False, name="walid"), 'laps':3,
})

tracks = ['abyss',
 'black_forest',
 'candela_city',
 'cocoa_temple',
 'cornfield_crossing',
 'fortmagma',
 'gran_paradiso_island',
 'hacienda',
 'lighthouse',
 'mines',
 'minigolf',
 'olivermath',
 'ravenbridge_mansion',
 'sandtrack',
 'scotland',
 'snowmountain',
 'snowtuxpeak',
 'stk_enterprise',
 'volcano_island',
 'xr591',
 'zengarden']

for i, venv in enumerate((noob_vec_env.envs)):
    venv.env.default_track = tracks[i%len(tracks)]

# expert = load_policy(
#     "ppo-huggingface",
#     organization="HumanCompatibleAI",
#     env_name="seals-CartPole-v0",
#     venv=noob_vec_env,
# )

# rollouts = rollout.rollout(
#     expert,
#     expert_vec_env,
#     rollout.make_sample_until(min_episodes=60),
#     rng=np.random.default_rng(SEED),
# )


..:: Antarctica Rendering Engine 2.0 ::..


In [2]:
from stk_actor.replay_buffer import SACRolloutBuffer
import joblib, torch 

buffer1 = joblib.load('all_tracks_buffer_steps_2mil')

..:: Antarctica Rendering Engine 2.0 ::..
..:: Antarctica Rendering Engine 2.0 ::..
..:: Antarctica Rendering Engine 2.0 ::..
..:: Antarctica Rendering Engine 2.0 ::..
..:: Antarctica Rendering Engine 2.0 ::..
..:: Antarctica Rendering Engine 2.0 ::..
..:: Antarctica Rendering Engine 2.0 ::..
..:: Antarctica Rendering Engine 2.0 ::..
..:: Antarctica Rendering Engine 2.0 ::..
..:: Antarctica Rendering Engine 2.0 ::..
..:: Antarctica Rendering Engine 2.0 ::..
..:: Antarctica Rendering Engine 2.0 ::..
..:: Antarctica Rendering Engine 2.0 ::..
..:: Antarctica Rendering Engine 2.0 ::..
..:: Antarctica Rendering Engine 2.0 ::..
..:: Antarctica Rendering Engine 2.0 ::..
..:: Antarctica Rendering Engine 2.0 ::..
..:: Antarctica Rendering Engine 2.0 ::..
..:: Antarctica Rendering Engine 2.0 ::..
..:: Antarctica Rendering Engine 2.0 ::..


In [3]:
import numpy as np
import dataclasses
from typing import Optional, List, Dict, Any, NamedTuple
import torch

def get_trajectories_from_buffer(buffer: SACRolloutBuffer) -> List[TrajectoryWithRew]:
    """
    Convert buffer data into a list of TrajectoryWithRew objects.
    Each trajectory represents a complete episode or episode fragment.
    
    Args:
        buffer: SACRolloutBuffer containing the transitions
        
    Returns:
        List of TrajectoryWithRew objects
    """
    trajectories = []
    current_trajectory = {
        'obs': [],
        'acts': [],
        'rews': [],
        'infos': [],
        'steps': []
    }
    
    # Convert buffer data to numpy for processing
    observations = buffer.observations.numpy()
    actions = np.stack([act.numpy() for act in buffer.actions], axis=1)
    rewards = buffer.rewards.numpy()
    dones = buffer.dones.numpy()
    steps = buffer.steps.numpy()
    next_observations = buffer.next_observations.numpy()
    
    # Process each transition in the buffer
    for i in range(buffer.size):
        # Check if this is the start of a new episode (step == 0)
        if steps[i][0] == 0 and len(current_trajectory['obs']) > 0:
            print(i, steps[i][0],)
            # Convert current trajectory to TrajectoryWithRew and add to list
            current_trajectory['obs'].append(next_observations[i-1])
            trajectories.append(_create_trajectory(
                current_trajectory['obs'],
                current_trajectory['acts'],
                current_trajectory['rews'],
                current_trajectory['infos'],
                True  # Terminal since it's end of episode
            ))
            
            # Reset current trajectory
            current_trajectory = {
                'obs': [],
                'acts': [],
                'rews': [],
                'infos': [],
                'steps': []
            }
            
        # Add current transition to trajectory
        current_trajectory['obs'].append(observations[i])
        current_trajectory['acts'].append(actions[i])
        current_trajectory['rews'].append(rewards[i])
        current_trajectory['infos'].append({
            'step': steps[i],
            'done': False
        })
        current_trajectory['steps'].append(steps[i])
        
        # If this is the last transition or it's done, create a trajectory
        if i == buffer.size - 1:
            
            if len(current_trajectory['obs']) > 0:
                # Add final observation
                if i + 1 < buffer.size:
                    current_trajectory['obs'].append(observations[i + 1])
                else:
                    current_trajectory['obs'].append(observations[i])  # Use last observation if at buffer end
                    
                trajectories.append(_create_trajectory(
                    current_trajectory['obs'],
                    current_trajectory['acts'],
                    current_trajectory['rews'],
                    current_trajectory['infos'],
                    True
                ))
                # Reset current trajectory
                current_trajectory = {
                    'obs': [],
                    'acts': [],
                    'rews': [],
                    'infos': [],
                    'steps': []
                }

    return trajectories

def _create_trajectory(
    observations: List[np.ndarray],
    actions: List[np.ndarray],
    rewards: List[np.ndarray],
    infos: List[Dict[str, Any]],
    terminal: bool
) -> TrajectoryWithRew:
    """
    Create a TrajectoryWithRew object from lists of transitions.
    
    Args:
        observations: List of observations including final observation
        actions: List of actions
        rewards: List of rewards
        infos: List of info dictionaries
        terminal: Whether trajectory ends in terminal state
        
    Returns:
        TrajectoryWithRew object
    """
    return TrajectoryWithRew(
        obs=np.stack(observations),
        acts=np.stack(actions),
        rews=np.stack(rewards).flatten(),
        infos=np.array(infos),
        terminal=terminal
    )

def get_trajectories_by_track(
    buffer: SACRolloutBuffer,
    track_name: str
) -> List[TrajectoryWithRew]:
    """
    Get trajectories for a specific track from the buffer.
    
    Args:
        buffer: SACRolloutBuffer containing the transitions
        track_name: Name of the track to filter trajectories for
        
    Returns:
        List of TrajectoryWithRew objects for the specified track
    """
    track_id = buffer.tracks[track_name]
    track_mask = buffer.track.numpy().flatten() == track_id
    
    # Create a new temporary buffer with only the track data
    temp_buffer = SACRolloutBuffer(
        buffer_size=buffer.buffer_size,
        obs_dim=buffer.obs_dim,
        action_dims=buffer.action_dims
    )
    
    # Copy only the track data
    temp_buffer.observations = buffer.observations[track_mask]
    temp_buffer.actions = [act[track_mask] for act in buffer.actions]
    temp_buffer.rewards = buffer.rewards[track_mask]
    temp_buffer.next_observations = buffer.next_observations[track_mask]
    temp_buffer.dones = buffer.dones[track_mask]
    temp_buffer.steps = buffer.steps[track_mask]
    temp_buffer.track = buffer.track[track_mask]
    temp_buffer.size = track_mask.sum()
    
    return get_trajectories_from_buffer(temp_buffer)

In [4]:
rollouts = get_trajectories_from_buffer(buffer1)

2871 0.0
8764 0.0
11182 0.0
13913 0.0
16844 0.0
19281 0.0
22393 0.0
25023 0.0
26873 0.0
29509 0.0
31222 0.0
32247 0.0
35328 0.0
38023 0.0
40714 0.0
43290 0.0
45436 0.0
48577 0.0
53313 0.0
56087 0.0
57575 0.0
59483 0.0
63249 0.0
64792 0.0
66589 0.0
68506 0.0
70078 0.0
72056 0.0
73768 0.0
75066 0.0
76800 0.0
78016 0.0
78798 0.0
80884 0.0
82601 0.0
84393 0.0
85986 0.0
87402 0.0
89409 0.0
92467 0.0
94302 0.0
95366 0.0
97020 0.0
100531 0.0
101905 0.0
103579 0.0
105259 0.0
106693 0.0
108564 0.0
110121 0.0
111213 0.0
112805 0.0
113931 0.0
114612 0.0
116455 0.0
118015 0.0
119590 0.0
121027 0.0
122352 0.0
124285 0.0
127091 0.0
128809 0.0
129766 0.0
132632 0.0
138414 0.0
140773 0.0
143489 0.0
146382 0.0
148859 0.0
152017 0.0
154648 0.0
156532 0.0
159178 0.0
160921 0.0
161955 0.0
165025 0.0
167795 0.0
170494 0.0
173087 0.0
175256 0.0
178385 0.0
183115 0.0
185866 0.0
187325 0.0
189200 0.0
193037 0.0
194773 0.0
196486 0.0
198380 0.0
199930 0.0
201968 0.0
203700 0.0
204986 0.0
206713 0.0
207882 0.0


In [5]:
len(rollouts)

315

In [6]:
from stk_actor.agent import UnifiedSACPolicy


net_arch=[512,512,512,512]
activation_fn=torch.nn.SiLU
filename = 'policy_512_512_512_512_SiLU_3_statedict'

learner = PPO(
    env=noob_vec_env,
    policy=MlpPolicy,
    batch_size=64,
    ent_coef=0.0,
    learning_rate=0.0005,
    gamma=0.95,
    clip_range=0.1,
    vf_coef=0.1,
    n_epochs=5,
    seed=SEED,
    policy_kwargs=dict(net_arch=net_arch, activation_fn=activation_fn,), 
)

action_dims = [space.n for space in noob_vec_env.action_space]
unified_policy = UnifiedSACPolicy(
    noob_vec_env.observation_space, 
    action_dims, 
    net_arch=net_arch, 
    activation_fn=activation_fn
)
unified_policy.load_state_dict(torch.load(filename))
learner.policy.load_state_dict(unified_policy.shared.state_dict())

/var/folders/5b/wsyz4crx22bg_7tpsgbw3tzw0000gn/T/ipykernel_64908/3874999844.py:29: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  unified_policy.load_state_dict(torch.load(fi

<All keys matched successfully>

In [7]:

# reward_net = BasicShapedRewardNet(
#     observation_space=noob_vec_env.observation_space,
#     action_space=noob_vec_env.action_space,
#     normalize_input_layer=RunningNorm,
# )
# airl_trainer = AIRL(
#     demonstrations=rollouts,
#     demo_batch_size=2048,
#     gen_replay_buffer_capacity=512,
#     n_disc_updates_per_round=16,
#     venv=noob_vec_env,
#     gen_algo=learner,
#     reward_net=reward_net,
#     allow_variable_horizon=True
# )


density_trainer = db.DensityAlgorithm(
    venv=noob_vec_env,
    rng=np.random.default_rng(0),
    demonstrations=rollouts,
    rl_algo=learner,
    density_type=db.DensityType.STATE_ACTION_DENSITY,
    is_stationary=True,
    kernel="gaussian",
    kernel_bandwidth=0.4,
    standardise_inputs=True,
)

In [8]:
density_trainer.train()

In [10]:
import pprint


def print_stats(density_trainer, n_trajectories):
    stats = density_trainer.test_policy(n_trajectories=n_trajectories)
    print("True reward function stats:")
    pprint.pprint(stats)
    stats_im = density_trainer.test_policy(true_reward=False, n_trajectories=n_trajectories)
    print("Imitation reward function stats:")
    pprint.pprint(stats_im)

In [ ]:
print("Stats before training:")
print_stats(density_trainer, 1)

In [11]:
density_trainer.train_policy(100)  # Train for 1_000_000 steps to approach expert performance.

ValueError: Episodes of different length detected: {1040, 1500, 1231}. Variable horizon environments are discouraged -- termination conditions leak information about reward. See https://imitation.readthedocs.io/en/latest/getting-started/variable-horizon.html for more information. If you are SURE you want to run imitation on a variable horizon task, then please pass in the flag: `allow_variable_horizon=True`.

In [ ]:




print("Stats after training:")
print_stats(density_trainer, 1)